# Program 2

In [1]:
import csv
#initiate the most generic hypothesis
def g_0(n):
    return ("?",)*n
#initiate the most specific hypothesis
def s_0(n):
    return ('ɸ',)*n


In [2]:
def more_general(h,ex):
    # h1 = hypothesis 
    # h2 = examples
    # print('hypothessis = ',h1,'\nexamples = ',examples)
    more_general_parts = []
    #each attribute value of h1 and h2 is stored in x and y respectively
    for x, y in zip(h,ex):
        
        mg = x == "?" or (x != "ɸ" and (x == y or y == "ɸ"))
        #append true to the list more_general_parts if x is more generic than y
        more_general_parts.append(mg)
    #returns true only if all the values of the list are true 
    return all(more_general_parts)



In [3]:
def fulfills(example , hypothesis):
    return more_general(hypothesis , example)


In [4]:
def min_generalizations(h, x):
    h_new = list(h)
    for i in range(len(h)):
        #checks if hypothesis is not more general than the training example
        if not fulfills(x[i], h[i]):
            #if hypthesis contains 'ɸ' replace with the value in the training ex else replace with '?'
            h_new[i] = '?' if h[i] != 'ɸ' else x[i]
    # return the new hypthesis list with commmas between each attribute value
    return [tuple(h_new)]

In [5]:
def min_specializations(h, domains, x):
    results = []
    #if hypthesis contains '?' then specialize h with all possible values of an attribute apart from the one in training ex
    #if training ex contains rainy,then update h with sunny
    for i in range(len(h)):
        if h[i] == "?":
            #all possible values for an attribute
            for val in domains[i]:
                #checks if instance attribute value is not same as the domain attribute value
                if x[i] != val:
                    #update only ith attribute values in h, remaining attribute values remains same
                    h_new = h[:i] + (val,   ) + h[i+1:]            
        elif h[i]!= "ɸ":
            #if hypthesis does not contain '?' and 'ɸ' then specialize h with 'ɸ'
            h_new = h[:i] + ('ɸ',) + h[i+1:]
        results.append(h_new)
    return results


In [6]:

try:
    csvFile = open('CandidateElimination1.csv')
except:
    print('file not found')
    exit()
examples = [tuple(line) for line in csv.reader(csvFile)]
examples

[('sunny', 'warm', 'normal', 'strong', 'warm', 'same', 'yes'),
 ('sunny', 'warm', 'high', 'strong', 'warm', 'same', 'yes'),
 ('rainy', 'cold', 'high', 'strong', 'warm', 'change', 'no'),
 ('sunny', 'warm', 'high', 'strong', 'cool', 'change', 'yes')]

In [7]:
#get the possible values of an attribute in the training example
def get_domains(examples):
    #initialize d as an empty set of values for each attribute in the instance
    print(examples)
    d = [set() for i in examples[0]]
    print('d = ',d)
    for x in examples:
        #enumerate function adds a counter to the collection 
        print('enumerate x = ',list(enumerate(x)))
        for i, xi in enumerate(x):
            #add the value of an instance attribute to d
            d[i].add(xi)
    #return a list of list of sorted values of each attribute
    print('d unsorted = ',d)
    print('sorted tuples = ',[sorted(x) for x in d])
    return [list(sorted(x)) for x in d]

get_domains(examples)

[('sunny', 'warm', 'normal', 'strong', 'warm', 'same', 'yes'), ('sunny', 'warm', 'high', 'strong', 'warm', 'same', 'yes'), ('rainy', 'cold', 'high', 'strong', 'warm', 'change', 'no'), ('sunny', 'warm', 'high', 'strong', 'cool', 'change', 'yes')]
d =  [set(), set(), set(), set(), set(), set(), set()]
enumerate x =  [(0, 'sunny'), (1, 'warm'), (2, 'normal'), (3, 'strong'), (4, 'warm'), (5, 'same'), (6, 'yes')]
enumerate x =  [(0, 'sunny'), (1, 'warm'), (2, 'high'), (3, 'strong'), (4, 'warm'), (5, 'same'), (6, 'yes')]
enumerate x =  [(0, 'rainy'), (1, 'cold'), (2, 'high'), (3, 'strong'), (4, 'warm'), (5, 'change'), (6, 'no')]
enumerate x =  [(0, 'sunny'), (1, 'warm'), (2, 'high'), (3, 'strong'), (4, 'cool'), (5, 'change'), (6, 'yes')]
d unsorted =  [{'rainy', 'sunny'}, {'warm', 'cold'}, {'normal', 'high'}, {'strong'}, {'warm', 'cool'}, {'same', 'change'}, {'no', 'yes'}]
sorted tuples =  [['rainy', 'sunny'], ['cold', 'warm'], ['high', 'normal'], ['strong'], ['cool', 'warm'], ['change', 'sa

[['rainy', 'sunny'],
 ['cold', 'warm'],
 ['high', 'normal'],
 ['strong'],
 ['cool', 'warm'],
 ['change', 'same'],
 ['no', 'yes']]

In [8]:
def candidate_elimination(examples):
    #domains contains values of all attributes except target attribute
    domains = get_domains(examples)[:-1]
    #initililaze the most generic and specific hypothesis depending on the no. of attributes 
    G = set([g_0(len(domains))])
    S = set([s_0(len(domains))])
    i = 0
    print("\n G[{0}]:".format(i), G)
    print("\n S[{0}]:".format(i), S)
    for xcx in examples:
        i = i + 1
        x, cx = xcx[:-1], xcx[-1]  # Splitting data into attributes and decisions
        if cx == 'Y':  # x is positive example
            G = {g for g in G if fulfills(x, g)}
            S = generalize_S(x, G, S)
        else:  # x is negative example
            S = {s for s in S if not fulfills(x, s)}
            G = specialize_G(x, domains, G, S)
        print("\n G[{0}]:".format(i), G)
        print("\n S[{0}]:".format(i), S)
    return


In [9]:
def generalize_S(x, G, S):
    S_prev = list(S)
    for s in S_prev:
         if not fulfills(x, s):
            S.remove(s)
            Splus = min_generalizations(s, x)
            ## keep only generalizations that have a counterpart in G
            S.update([h for h in Splus if any([more_general(g,h)
                                               for g in G])])
            ## remove hypotheses less specific than any other in S
            S.difference_update([h for h in S if
                                 any([more_general(h, h1)
                                      for h1 in S if h != h1])])
    return S

In [10]:
def specialize_G(x, domains, G, S):
    G_prev = list(G)
    for g in G_prev:
          if fulfills(x, g):
            G.remove(g)
            Gminus = min_specializations(g, domains, x)
            ## keep only specializations that have a conuterpart in S
            G.update([h for h in Gminus if any([more_general(h, s)
                                                for s in S])])
            ## remove hypotheses less general than any other in G
            G.difference_update([h for h in G if
                                 any([more_general(g1, h)
                                      for g1 in G if h != g1])])
    return G


In [11]:
candidate_elimination(examples)

[('sunny', 'warm', 'normal', 'strong', 'warm', 'same', 'yes'), ('sunny', 'warm', 'high', 'strong', 'warm', 'same', 'yes'), ('rainy', 'cold', 'high', 'strong', 'warm', 'change', 'no'), ('sunny', 'warm', 'high', 'strong', 'cool', 'change', 'yes')]
d =  [set(), set(), set(), set(), set(), set(), set()]
enumerate x =  [(0, 'sunny'), (1, 'warm'), (2, 'normal'), (3, 'strong'), (4, 'warm'), (5, 'same'), (6, 'yes')]
enumerate x =  [(0, 'sunny'), (1, 'warm'), (2, 'high'), (3, 'strong'), (4, 'warm'), (5, 'same'), (6, 'yes')]
enumerate x =  [(0, 'rainy'), (1, 'cold'), (2, 'high'), (3, 'strong'), (4, 'warm'), (5, 'change'), (6, 'no')]
enumerate x =  [(0, 'sunny'), (1, 'warm'), (2, 'high'), (3, 'strong'), (4, 'cool'), (5, 'change'), (6, 'yes')]
d unsorted =  [{'rainy', 'sunny'}, {'warm', 'cold'}, {'normal', 'high'}, {'strong'}, {'warm', 'cool'}, {'same', 'change'}, {'no', 'yes'}]
sorted tuples =  [['rainy', 'sunny'], ['cold', 'warm'], ['high', 'normal'], ['strong'], ['cool', 'warm'], ['change', 'sa